[View in Colaboratory](https://colab.research.google.com/github/suneetsawant/nlp/blob/master/Toxic_Comments.ipynb)

# Import all the libraries 

In [1]:
import pandas as pd 
import numpy as np 
import os,shutil
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import CuDNNLSTM,Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.models import Model,Sequential
import seaborn as sns
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


%reload_ext autoreload
%autoreload 2

Using TensorFlow backend.


# Set up pydrive for uploading and downloading files (model and weights) 

The ***Drive()*** class gives method ***fileaction()***  to either upload or download a list of files 

### Usage :  
      
     To upload files named f1,f2  
        Drive().fileaction([f1,f2],'up')
     Similarly to download  
        Drive().fileaction([f1,f2],'down')

In [0]:
# Authenticate and create the PyDrive client.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
class Drive(): 
  



  
  def __init__(self) :  
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    self.drive = GoogleDrive(gauth)


  def fileaction(self,files,op='up') : 
    file_list = self.drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
    for filename in files:
      flag = 0 

      for file1 in file_list:
        if (file1['title']) == filename :
            if (op == 'up'and flag==0) : 
                file1.Delete()
                self.upload(filename)
                flag = 1

            elif (op == 'down') : 
                self.download(filename,file1) 

      if(op=='up' and flag==0): 
            self.upload(filename)
            flag = 1

  def upload(self,filename) : 
      Uploadfile = self.drive.CreateFile({'title': filename})
      Uploadfile.SetContentFile(filename)
      Uploadfile.Upload()
      print("Saved '{}' to Drive".format(filename))

  def download(self,filename,file1): 
      downloaded = self.drive.CreateFile({'id':file1['id']})
      downloaded.GetContentFile(filename)
      print("Downloaded '{}' from Drive".format(filename))

# **Download the Dataset**

In [4]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1523554043&Signature=jxNQ%2BF1ridd29nYJPxVMQo3YM6axdT%2Bok0D0mCoQOFzCfpk8%2FJ1zF0jcpuD1Dy1gJbOfDnz2ndX9Aak4Hyc%2BZrmQCJRMSS3WwWg43%2B3AH99UntZk%2FHOmiEKCPFn5miNGe%2BTB4E6mfraJBtT7XBmX7vnye%2BOIhC1tpxFo6tpDPltgi6SZmmLMO76vXwKC9KU7eWguxnS9VTLuS4O1NK8cdwW2jucuQT8OXBu6C8rnfhpFb4wPqnUK7TXf80kd4zRNsAjy3czFP54pwBV9whBceIxW5arzdC4H2AJcPNYgRu4PQZ3i1niR1VpjuPTCCEb6xN0hmkeSFThLb2BK4mDmkg%3D%3D" -O "train.csv.zip" -c
!yes | unzip train.csv.zip 

   

--2018-04-12 17:17:24--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1523554043&Signature=jxNQ%2BF1ridd29nYJPxVMQo3YM6axdT%2Bok0D0mCoQOFzCfpk8%2FJ1zF0jcpuD1Dy1gJbOfDnz2ndX9Aak4Hyc%2BZrmQCJRMSS3WwWg43%2B3AH99UntZk%2FHOmiEKCPFn5miNGe%2BTB4E6mfraJBtT7XBmX7vnye%2BOIhC1tpxFo6tpDPltgi6SZmmLMO76vXwKC9KU7eWguxnS9VTLuS4O1NK8cdwW2jucuQT8OXBu6C8rnfhpFb4wPqnUK7TXf80kd4zRNsAjy3czFP54pwBV9whBceIxW5arzdC4H2AJcPNYgRu4PQZ3i1niR1VpjuPTCCEb6xN0hmkeSFThLb2BK4mDmkg%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.128, 2607:f8b0:400e:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27619914 (26M) [application/zip]
Saving to: ‘train.csv.zip’

train.csv.zip       100%[===================>]  26.34M  76.5MB/s    in 0.3s    

2018-04-12 17:17:25 (76.5 MB/s) - ‘train.csv

# Data Preprocessing 

In [0]:
  def prepareData(shuffle,valid_ratio) :
      
      dtrain = pd.read_csv('train.csv')
      
      dftrain, dfval = train_test_split(dtrain, test_size= valid_ratio,shuffle=shuffle)

      classes = dftrain.columns[2:]
      Xtrain = dftrain['comment_text'] 
      Ytrain = dftrain[classes].values 

      Xval = dfval['comment_text'] 
      Yval = dfval[classes].values 
      
      return Xtrain,Ytrain,Xval,Yval
    
  def createTokens(vocab_size, data) :
      tokenizer = Tokenizer(num_words = vocab_size)
      tokenizer.fit_on_texts(data)
      return tokenizer
  
  def tokenToSequence(data,tokenzier,max_len_sentence): 
      data = tokenizer.texts_to_sequences(data)
      data = pad_sequences(data,maxlen=max_len_sentence)
      return data   

# Define the Model 

In [0]:
  def Model(restore,loss,optimizer): 
      if(restore): return restoreModel(loss,optimizer)
      else : return newModel(loss,optimizer)
 

  def newModel(loss,optimizer) : 
      model = Sequential()
      model.add(Embedding(vocab_size,256,input_length=max_len_sentence,name='Embedding'))
      model.add(CuDNNLSTM(75,name ="lstm")) 
      
      model.add(Dense(100,activation='relu',name='Dense1'))
      model.add(Dropout(0.2,name='Dropout1')) 
      
      model.add(Dense(6,activation='sigmoid',name='Dense2'))
      model.compile (loss=loss,
                        optimizer=optimizer)
      return model 
  
  def restoreModel(loss,optimizer): 
      
      Drive().fileaction(['model.json','weights.hdf5'],'down') 
      with open('model.json', 'r') as json_file:
          loaded_model_json = json_file.read()
      model = model_from_json(loaded_model_json)
      model.compile (loss=loss,
                        optimizer=optimizer)
      # load weights into new model
      model.load_weights("weights.hdf5")
      print("Loaded saved model")
      
      return model
    
  def buildModel(X,Y,epochs,batchsize,loss,optimizer,restore): 
    
      model = Model(restore,loss,optimizer)      
      filepath = "weights.hdf5"
      checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=1, 
                                   save_best_only=True, mode='auto')
      callbacks_list = [checkpoint]

      model.fit(X,Y, batch_size=batch_size, epochs=epochs,callbacks=callbacks_list,verbose=1)

      model_json = model.to_json()
      with open("model.json", "w") as json_file:
          json_file.write(model_json) 

      Drive().fileaction(['model.json','weights.hdf5'],'up') 
      return model
    
  def trainModel(Xtrain,Ytrain,Xval,Yval,epochs,batchsize,loss,optimizer,restore): 
       
      if(restore==1) : flag = 1 
      else: flag = 0    

      for i in range(epochs) :
        
        if(flag==0 and i>0): flag = 1
        model = buildModel(Xtrain,Ytrain,1,batchsize,loss,optimizer,restore=flag)
        trainScore = evaluateModel(Xtrain,Ytrain,model) 
        validScore = evaluateModel(Xval,Yval,model)
        print('Training Score : {} - Validation Score:{}'.format(trainScore,validScore))
      
      model.summary()
      print('Model is evaluated on metric ROC AUC')

      return model
    
  def evaluateModel(X,Y,model): 
      preds = model.predict(X,verbose=1)
      return roc_auc_score (Y,preds)
    
 


# Training Model

In [48]:
batch_size = 32
epochs = 5
loss='binary_crossentropy'
optimizer='adam'
restore = 1
shuffle = True
valid_ratio = 0.2
vocab_size = 20000
max_len_sentence = 200 

Xtrain,Ytrain,Xval,Yval = prepareData(shuffle,valid_ratio)

tokenizer = createTokens(vocab_size,Xtrain)

Xtrain =   tokenToSequence(Xtrain,tokenizer,max_len_sentence)
Xval   =   tokenToSequence(Xval,tokenizer,max_len_sentence)



model = trainModel(Xtrain,Ytrain,Xval,Yval,epochs,batch_size,loss, optimizer,restore)


Downloaded 'model.json' from Drive
Downloaded 'weights.hdf5' from Drive
Loaded saved model
Epoch 1/1
 36544/127656 [=======>......................] - ETA: 1:51 - loss: 0.1015

127656/127656 [==============================] - 152s 1ms/step - loss: 0.0701



Epoch 00001: loss improved from inf to 0.07009, saving model to weights.hdf5
Saved 'model.json' to Drive
Saved 'weights.hdf5' to Drive
 66784/127656 [==============>...............] - ETA: 25s

31915/31915 [==============================] - 14s 424us/step
Training Score : 0.9858753898228395 - Validation Score:0.977025700566372
Downloaded 'model.json' from Drive
Downloaded 'weights.hdf5' from Drive
Loaded saved model
Epoch 1/1
  7104/127656 [>.............................] - ETA: 2:48 - loss: 0.0402

127656/127656 [==============================] - 152s 1ms/step - loss: 0.0424


...


Epoch 00001: loss improved from inf to 0.02848, saving model to weights.hdf5
Saved 'model.json' to Drive
Saved 'weights.hdf5' to Drive
 65920/127656 [==============>...............] - ETA: 26s

31915/31915 [==============================] - 14s 429us/step
Training Score : 0.9970893955034238 - Validation Score:0.9796250882099713
Downloaded 'model.json' from Drive
Downloaded 'weights.hdf5' from Drive
Loaded saved model
Epoch 1/1
  7008/127656 [>.............................] - ETA: 2:53 - loss: 0.0193

127656/127656 [==============================] - 153s 1ms/step - loss: 0.0238



Epoch 00001: loss improved from inf to 0.02376, saving model to weights.hdf5
Saved 'model.json' to Drive
Saved 'weights.hdf5' to Drive
 66016/127656 [==============>...............] - ETA: 26s

31915/31915 [==============================] - 14s 427us/step
Training Score : 0.9979360827714366 - Validation Score:0.978293122911761
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding (Embedding)        (None, 200, 256)          5120000   
_________________________________________________________________
lstm (CuDNNLSTM)             (None, 75)                99900     
_________________________________________________________________
Dense1 (Dense)               (None, 100)               7600      
_________________________________________________________________
Dropout1 (Dropout)           (None, 100)               0         
_________________________________________________________________
Dense2 (Dense)               (None, 6)                 606       
Total params: 5,228,106
Trainable params: 5,228,106
Non-trainable params: 0
_______________________________________________________________

In [0]:
xtest = tokenizer.texts_to_sequences(Xtrain[2001:2010,])
xtest = pad_sequences(xtest,maxlen=max_len_sentence) 

ytest = Ytrain[2001:2010,:]

v =50
xtest = X[:5000,:]
ytest = Y[:5000,:]

preds = model.predict(xtest)


# Test the model

In [118]:
!wget wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1523721107&Signature=eJccX9d54GdNDdg%2B4iyiHf9L0SuOoej9SIDq2BRRIWL3V8X0l4%2FVXNxDLOAoxgOXNT7829tp2T%2BjMjoFVOABNdiqYeshPMFVK0Z%2B1bR%2BmolHAqzdpzdUnlCEkLjLEHc2l3DRscDLxxmXCZiPr1Cw3r%2F%2FPBT%2FfjHkNQvHCklaceeb7phAYj%2BEweYwtRJ%2B25F83ysn1BFalbT7IxvQax4MRzFdKs6%2F4X5bjPIZxViq1xIqrk9p1TwGgLwTUaAdtH0Sgsd6ne4sacfyKT6zREIWbkNNU0U%2BKHDStV3vo4VgB7V04nL9LvYPU6KAGKXKuqKDps7tTH3YgJ2sdAJB9tnn1w%3D%3D" -O "test.csv.zip" -c
!unzip -o test.csv.zip
dtest = pd.read_csv('test.csv')
Xtest = dtest['comment_text']
dtest.info()


--2018-04-11 18:23:56--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2018-04-11 18:23:56--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1523721107&Signature=eJccX9d54GdNDdg%2B4iyiHf9L0SuOoej9SIDq2BRRIWL3V8X0l4%2FVXNxDLOAoxgOXNT7829tp2T%2BjMjoFVOABNdiqYeshPMFVK0Z%2B1bR%2BmolHAqzdpzdUnlCEkLjLEHc2l3DRscDLxxmXCZiPr1Cw3r%2F%2FPBT%2FfjHkNQvHCklaceeb7phAYj%2BEweYwtRJ%2B25F83ysn1BFalbT7IxvQax4MRzFdKs6%2F4X5bjPIZxViq1xIqrk9p1TwGgLwTUaAdtH0Sgsd6ne4sacfyKT6zREIWbkNNU0U%2BKHDStV3vo4VgB7V04nL9LvYPU6KAGKXKuqKDps7tTH3YgJ2sdAJB9tnn1w%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 416 Requested range not satisfiable

    The file is

In [0]:

Xtest = tokenizer.texts_to_sequences(Xtest)
Xtest = pad_sequences(Xtest,maxlen=max_len_sentence) 

In [114]:
preds = model.predict(Xtest,verbose=1)
preds[preds>=0.5] = 1
preds[preds<0.5] = 0
preds.shape

153164/153164 [==============================] - 59s 388us/step


(153164, 6)

In [0]:
result = pd.DataFrame(preds,columns = classes)
result['id'] = dtest['id']
result.to_csv('result.csv',index=False)


In [116]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 7 columns):
toxic            153164 non-null float32
severe_toxic     153164 non-null float32
obscene          153164 non-null float32
threat           153164 non-null float32
insult           153164 non-null float32
identity_hate    153164 non-null float32
id               153164 non-null object
dtypes: float32(6), object(1)
memory usage: 4.7+ MB


In [0]:
from google.colab import files
files.download('result.csv')